In [72]:
import numpy as np
import math
import statsmodels.api as sm
import imblearn
from tqdm.notebook import tqdm
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite, community 
from numpy.random import choice
import itertools
pd.set_option('display.max_columns', None)

## Bipartite network generation

### Data preparation

In [73]:
cb = pd.read_csv('cb.csv')

In [65]:
cb.groupby(['target_continent','investor_continent']).size().unstack(fill_value=0)

investor_continent,Africa,Asia,Australia,Europe,North America,South America
target_continent,,,,,,
Africa,250,48,3,112,89,0
Asia,34,17058,35,705,2257,6
Australia,0,49,523,60,186,1
Europe,16,656,75,21472,3658,35
North America,45,3595,313,6264,75739,55
South America,0,14,4,99,182,464


In [66]:
cb_a = cb.groupby(['target_continent','investor_continent']).size().unstack(fill_value=0)
cb_a["sum"] = cb_a.sum(axis = 0)
cb_a = cb_a.loc[:,"Africa":"South America"].div(cb_a["sum"], axis=0)
cb_a

investor_continent,Africa,Asia,Australia,Europe,North America,South America
target_continent,,,,,,
Africa,0.724638,0.139130,0.008696,0.324638,0.257971,0.000000
Asia,0.001587,0.796359,0.001634,0.032913,0.105369,0.000280
Australia,0.000000,0.051417,0.548793,0.062959,0.195173,0.001049
Europe,0.000557,0.022848,0.002612,0.747841,0.127403,0.001219
North America,0.000548,0.043782,0.003812,0.076287,0.922398,0.000670
South America,0.000000,0.024955,0.007130,0.176471,0.324421,0.827094


In [67]:
cb

,target_name,stage,size_MLN,date,investors_name,url,description,industry,target_country,target_collections,target_status,target_ID,target_founded,round_simp,target_continent,target_state,target_city,target_street,target_people,target_competitors,investor_name,investor_url,investor_description,investor_country,investor_type,investor_deals_now,investor_companies,investor_deals_total,investor_exit_12,investor_ppl,investor_deals_12,investor_collections,investor_id,investor_industry,investor_state,investor_continent,investor_city,investor_founded,investor_street,date_dt,DATE,CPIAUCSL,size_real
0,Shionogi & Co.,IPO,NaN,05/16/1949,Public,shionogi.com,Shionogi & Co. (TYO: 4507) focuses on the rese...,Pharmaceuticals / Drugs,Japan,Pharma Startups,IPO / Went public,262824,1878.0,IPO,Asia,NaN,Osaka,"1-8, Doshomachi 3-chome Chuo-ku…",NaN,Sumitomo Dainippon Pharma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1949-05,1949-05,23.91,NaN
1,Astellas Pharma,IPO,NaN,05/16/1949,Public,astellas.com,Astellas Pharma (TSE: 4503) specializes in the...,Pharmaceuticals / Drugs,Japan,"Digital Therapeutics, Pharma Startups, Cancer ...",IPO / Went public,243168,1923.0,IPO,Asia,NaN,Tokyo,"2-5-1, Nihonbashi-Honcho, Chuo-ku…","Yoshihiko Hatanaka (CEO),Yoshirou Myyokawa (EV...",Sumitomo Dainippon Pharma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1949-05,1949-05,23.91,NaN
2,Sumitomo Dainippon Pharma,IPO,NaN,05/16/1949,Public,ds-pharma.com,Sumitomo Dainippon Pharma (TYO: 4506) is a pha...,Pharmaceuticals / Drugs,Japan,Digital Therapeutics and Pharma Startups…,IPO / Went public,447180,1897.0,IPO,Asia,NaN,Osaka,"6-8, Doshomachi 2-chome Chuo-ku…",Masayo Tada (CEO),"Astellas Pharma,Chugai Pharmaceutical,Daiichi ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1949-05,1949-05,23.91,NaN
3,Konica Minolta,IPO,NaN,05/16/1949,Public,konicaminolta.com,"Konica Minolta (コニカミノルタ) (TYO: 4902), founded ...",Medical Devices & Equipment,Japan,Conference Exhibitors,IPO / Went public,57191,1936.0,IPO,Asia,NaN,Tokyo,"JP Tower 2-7-2 Marunouchi, Chiyoda-ku…",Michael Mathé (SVP),Xerox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1949-05,1949-05,23.91,NaN
4,Takeda Pharmaceutical,IPO,NaN,05/16/1949,Public,takeda.com,Takeda Pharmaceutical (TYO: 4502) (NYSE: TAK) ...,Pharmaceuticals / Drugs,Japan,"Poop Tech, Pharma Startups, Rare Diseases, Sle...",IPO / Went public,373007,1781.0,IPO,Asia,NaN,Tokyo,"2-1-1 Nihonbashihonmachi, Chuo-ku…","Frank Morich (CEO),Yasuchika Hasegawa (CEO),Da...","Sumitomo Dainippon Pharma,Sunovion…",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1949-05,1949-05,23.91,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187799,Beijing Shouyi,Acquired,NaN,08/17/2021,Healgen,shouyi-biochem.com…,"Beijing Shouyi is a biomedical enterprise, whi...",Medical Devices & Equipment,China,NaN,Acquired,669346,1998.0,Acquisition,Asia,Beijing,Beijing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08,2021-08,274.00,NaN
187800,Centaur Labs,Series A,15.89,08/17/2021,Undisclosed Investors,centaurlabs.com,Centaur Labs enables customers to build AI alg...,Internet Software & Services,United States,"Digital Health, Artificial Intelligence ( AI )...",Alive / Active,377820,2017.0,Series A,North America,Massachusetts,Boston,"883 Boylston St, Suite 200…","Erik Duhaime (CEO),Zach Rausnitz (CTO)…",Aigorithm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08,2021-08,274.00,15890000.0
187801,Streamline Health Solutions,IPO,NaN,08/17/2021,Public,streamlinehealth.net,Streamline Health (NASDAQ: STRM) provides solu...,Healthcare Software,United States,NaN,IPO / Went public,439295,NaN,IPO,North America,Ohio,Cincinnati,10200 Alliance Road Suite 200…

In [80]:
nodes_org = cb[['target_name', 'target_country', 'target_continent', 'industry',
               'target_status','target_ID']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_country', 'investor_continent', 'investor_type',
                'investor_id']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country", 
                                      "target_continent": "continent", "industry":"industry",
                                     "target_ID":"id"})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'id' })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index
nodes

,node,country,continent,industry,target_status,id,bipartite
0,Shionogi & Co.,Japan,Asia,Pharmaceuticals / Drugs,IPO / Went public,0,1
1,Astellas Pharma,Japan,Asia,Pharmaceuticals / Drugs,IPO / Went public,1,1
2,Sumitomo Dainippon Pharma,Japan,Asia,Pharmaceuticals / Drugs,IPO / Went public,2,1
3,Konica Minolta,Japan,Asia,Medical Devices & Equipment,IPO / Went public,3,1
4,Takeda Pharmaceutical,Japan,Asia,Pharmaceuticals / Drugs,IPO / Went public,4,1
...,...,...,...,...,...,...,...
83253,Shenzhen Qianhai Beizeng Asset Management Co. ...,China,Asia,Venture Capital,NaN,83253,0
83254,Wuxi High-Tech District Seed Fund,China,Asia,Other,NaN,83254,0
83255,Edvantage Group,China,Asia,Corporation,NaN,83255,0
83256,Healgen,NaN,NaN,NaN,NaN,83256,0
